# [Introduction to Data Science](http://datascience-intro.github.io/1MS041-2023/)    
## 1MS041, 2023 
&copy;2023 Raazesh Sainudiin, Benny Avelin. [Attribution 4.0 International     (CC BY 4.0)](https://creativecommons.org/licenses/by/4.0/)

## Concentration and limits

Consider an i.i.d. sequence of random variables $X_1,\ldots,X_n$ each being Bernoulli($1/2$). Then the concept of concentration is telling us that

$$
    P\left ( \left | \frac{1}{n} \sum_{i=1}^n X_i - \mathbb{E}(X_i) \right | > \epsilon \right )
$$

gets smaller as $n$ gets larger. For instance, using Chebychevs inequality we get

$$
    P\left ( \left | \frac{1}{n} \sum_{i=1}^n X_i - \mathbb{E}(X_i) \right | > \epsilon \right ) \leq \frac{\mathbb{V}\left( \frac{1}{n} \sum_{i=1}^n X_i \right )}{\epsilon^2} = \frac{\mathbb{V}\left( X_0 \right )}{\epsilon^2 n}
$$

We can see that this is at least true in the simulation below

In [2]:
from ipywidgets import interact, IntSlider
from Utils import discrete_histogram
import numpy as np
@interact 
def concentration(n=IntSlider(1,1,50,5)):
    import matplotlib.pyplot as plt
    X = np.random.randint(0,2,size=(n,10000))*2-1
    means = np.mean(X,axis=0)
    print("P(mean > mu + 0.3 ) = %.5f <= Chebychev %.5f" % (np.mean(means > 0.3),1/(0.3**2*n)))
    discrete_histogram(means,normed=True)
    plt.xlim(-1,1)

# 分析一下这句的语法：
# print("P(mean > mu + 0.3 ) = %.5f <= Chebychev %.5f" % (np.mean(means > 0.3), 1 / (0.3**2 * n)))
# 1.P(mean > mu + 0.3 ) = %.5f <= Chebychev %.5f" 是一个包含占位符 %f 的字符串。
  # 这些占位符表示将来要在字符串中插入浮点数
# 2.百分号格式化：
  # %是字符串格式化的操作符，右边的元组 (np.mean(means > 0.3), 
  # 1 / (0.3**2 * n)) 包含了要插入占位符的值。
  # %.5f 表示浮点数的格式，保留小数点后五位。
# 3.插入值：
  # np.mean(means > 0.3) 是一个布尔表达式，它计算均值大于0.3的概率。
  # 这个值将插入到第一个 %f 的位置。
  # 1 / (0.3**2 * n) 是 Chebyshev 不等式计算的上界，表示均值大于0.3的概率上限。
  # 这个值将插入到第二个 %f 的位置
# 4.打印结果：
  # print() 函数用于将格式化的字符串打印到控制台。
#整句要表达的意思，包括了两个部分：
#实验结果：它计算了均值大于0.3的概率，
    #具体地说，np.mean(means > 0.3) 计算了样本均值中有多少比例大于0.3的值，
    #然后通过字符串格式化插入到消息中。
#理论上的上界：它使用chebychev不等式计算了理论上的上界
    #具体来说，1 / (0.3**2 * n) 表示通过 Chebyshev 不等式得到的上界，
    #然后也通过字符串格式化插入到消息中
#整个消息的格式是："P(mean > mu + 0.3 ) = 实验结果 <= Chebychev 理论上的上界"，
    #这个消息告诉用户两个信息：实验中观察到的均值大于0.3的概率，
    #以及 Chebyshev 不等式提供的理论上的上界。
    #这有助于比较实际观察到的概率和理论上的概率上限。

interactive(children=(IntSlider(value=1, description='n', max=50, min=1, step=5), Output()), _dom_classes=('wi…

Hoeffdings inequality on the other hand is sharper and gives a bound much smaller, namely that

$$
    P\left ( \left |\frac{1}{n} \sum_{i=1}^n X_i - \mathbb{E}(X_i) \right | > \epsilon \right ) 
    \leq 2e^{-2n \epsilon^2}
$$

In [3]:
@interact 
def concentration(n=IntSlider(1,1,50,5)):
    import matplotlib.pyplot as plt
    X = np.random.randint(0,2,size=(n,10000))
    means = np.mean(X,axis=0)
    print("P(mean > mu + 0.3 ) = %.5f <= Hoeffding %.5f" % (np.mean(means > 0.5+0.3),np.exp(-2*n*0.3**2)))
    discrete_histogram(means,normed=True)
    plt.xlim(0,1)

interactive(children=(IntSlider(value=1, description='n', max=50, min=1, step=5), Output()), _dom_classes=('wi…

This is much closer, and in fact very close.

### Using concentration as a measure of confidence

We can use concentration as a measure of confidence in the following way. Consider $X_1,\ldots, X_n$ being i.i.d. sequence of Bernoulli($p$) for some unknown $p$. From the concept of concentration, we would expect that if we have many observations ($n$ large) we could use the empirical mean of the observations as a guess, but note that there is some variability as we saw in the above simulations. So what do we do? We use the concentration inequality to get information how far we can deviate from $p$ in the following way

$$
    P(\bar X_n - \mathbb{E}(\bar X_n) \geq \epsilon) \leq e^{-2n\epsilon^2}
$$

Since $\mathbb{E}(\bar X_n) = p$, rearrange and get

$$
    P(p \leq \bar X_n - \epsilon) \leq e^{-2n\epsilon^2}
$$

The complementary event thus satisfies

$$
    P(\bar X_n - \epsilon < p) \geq 1-e^{-2n\epsilon^2}
$$

We can do the same for the other side (see lecture notes) and we get

$$
    P(\bar X_n - \epsilon < p < \bar X_n + \epsilon) \geq 1-2 e^{-2n\epsilon^2}.
$$

If you where now asked to estimate $p$ using $n$ observations and give an interval where you with at least 95% confidence can say contains $p$, then you need to choose $\epsilon > 0$ such that

$$
    1-2 e^{-2n\epsilon^2} \geq 0.95.
$$

Smaller $\epsilon$ gives smaller intervals, so lets choose to have the smallest possible $\epsilon$ while still obaying the inequality above, i.e. we choose $\epsilon$ to solve

$$
    1-2 e^{-2n\epsilon^2} = 0.95.
$$

Rearranging we and taking log and then square root we obtain

$$
    \epsilon = \sqrt{-\frac{1}{2n}\ln\left(\frac{1-0.95}{2}\right)}
$$

In [5]:
from ipywidgets import FloatSlider
@interact #引入交互式函数
def concentration(n=IntSlider(value=1,min=50,max=500,step=50),p=FloatSlider(value=0.5, min=0,max=1,step=0.1)):
    X = np.random.binomial(1,p,size=(n))
    means = np.mean(X,axis=0)
    epsilon = np.sqrt(-1/(2*n)*np.log((1-0.95)/2))
    #print("95%% confidence interval [%.2f, %.2f] for p=%.2f n=%d" % (means-epsilon,means+epsilon,p,n))
    print(f"95% confidence interval [{means-epsilon:.2f}, {means+epsilon:.2f}] for p={p:.2f} n={n}")
    #用f-string语法更为简单清晰
#在 f-string 中，花括号 {} 用来标记将要插入的变量或表达式。
#在花括号中，可以使用变量、表达式或函数等。
#在花括号中使用冒号 : 后，可以指定格式化的方式。例如，.2f 表示保留两位小数。


interactive(children=(IntSlider(value=50, description='n', max=500, min=50, step=50), FloatSlider(value=0.5, d…

In [11]:
from ipywidgets import FloatSlider
@interact 
def concentration_300(n=IntSlider(value=1,min=50,max=5000,step=50)):
    X = np.minimum(np.abs(70+6*np.random.normal(size=(n))),300)
    #这一句的语法结构如下：
    #np.random.normal(size=(n)) 生成一个大小为 n 的数组，
    #其中的元素是从标准正态分布（均值为0，标准差为1）中抽取的随机数。
    #将上述随机数乘以6，然后加上70，得到一个服从均值为70、标准差为6的正态分布的数组
    #使用 np.abs 计算上述数组中每个元素的绝对值，确保所有的值都为非负数。
    #使用 np.minimum 函数将上述数组中的每个元素与300比较，保留较小的那个值。
    
    means = np.mean(X,axis=0)
    epsilon = np.sqrt(-1/(2*n/(300**2))*np.log((1-0.95)/2))
    print("95%% confidence interval [%.2f, %.2f] for n=%d" % (means-epsilon,means+epsilon,n))
    print(f"95% confidence interval [{means-epsilon:.2f},{means+epsilon:.2f}] for n={n}")

interactive(children=(IntSlider(value=50, description='n', max=5000, min=50, step=50), Output()), _dom_classes…

These things are super useful. Lets go back to our sms spam/ham problem and see what we can say there

In [1]:
from Utils import load_sms 
sms_data = load_sms()
sms_data[:2]   #这里代表的是选取了sms_data的前两行

print(sms_data[:2])  

#若要取sms_data的前两列，语法为：sms_data[:,:2]

[('Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...', 0), ('Ok lar... Joking wif u oni...', 0)]


In [2]:
interesting_words=set(['free','prize'])   #创立一个集合，包含两个string单词
TF10 = {True: 1, False: 0}   #定义TF10字典，把布尔值映射为两个数字
Z_obs = [TF10[not interesting_words.isdisjoint([word.lower() for word in line[0].split(' ')])] for line in sms_data]
#对sms_data里的每行遍历，把每行的首个单词分离出来，并转换成小写
#看看这个分离出来的单词是否满足 不和interestingword交集为空，即和interestingword有交集
#按照TF10的映射，若有交集，布尔值为1，若无交集，布尔值为0，
#这些值一起组成一个列表，命名为Z_obs
#print(Z_obs)

#我感觉上面的语法有点拗口，双重否定。如果换一种语法：
M=[bool(set(interesting_words) & set([word.lower() for word in line[0].split(' ')]))for line in sms_data]
M_obs=[TF10[x] for x in M]

Y_obs = [y for x,y in sms_data]  # Y_obs 是 sms_data 里面的标签值

Recall that we computed the mean of Y_obs, which is the mean spam number. This is a Bernoulli random variable with unknown $p$, so we can use our methods above to compute a confidence interval

In [25]:
def epsilon_bernoulli(n,alpha):   #定义了一个epsilon_bernoulli函数，接受n和alpha两个参数
    return np.sqrt(-1/(2*n)*np.log((alpha)/2))   #返回的是 Bernoulli 分布的置信区间半径

In [27]:
epsilon = epsilon_bernoulli(len(Y_obs),0.05)   #n=len(Y_obs)，alpha=0.05，求epsilon
mean_Y_obs = np.mean(Y_obs)
print("[%.3f,%.3f]" % (mean_Y_obs-epsilon,mean_Y_obs+epsilon))   #%.3f 代表字符串保留三位小数

[0.116,0.152]


From this we get that we have provided a prediction as to what is the true probability of getting a spam email.

Lets take a look at the conditional probability. Recall that

$$
    P(Y = 1 \mid Z = 1) = \frac{P(Y = 1 \text{ and } Z = 1)}{P(Z = 1)}
$$

but this requires you to estimate both things on the right, and give a region for both and finally figure out an interval for the ratio. But, there is an easier way, which sometimes works better. Namely, to look at the random variable $Y \mid (Z=1)$ which we do by filtering

In [30]:
Y_mid_Z1 = [y for z,y in zip(Z_obs,Y_obs) if z == 1]

Now for this you have a certain number of observations and we can use this instead as follows

In [32]:
epsilon = epsilon_bernoulli(len(Y_mid_Z1),0.05)
mean_Y_obs = np.mean(Y_mid_Z1)
print("[%.3f,%.3f]" % (mean_Y_obs-epsilon,mean_Y_obs+epsilon))

[0.726,0.898]


## Being clever with Bennetts inequality

In [35]:
def bennett_epsilon(n,b,sigma,alpha):
    import scipy.optimize as so
    h = lambda u: (1+u)*np.log(1+u)-u
    f = lambda epsilon: np.exp(-n*sigma**2/b**2*h(b*epsilon/sigma**2))-alpha/2
    ans = so.fsolve(f,0.002)
    epsilon = np.abs(ans[0])
    print("Numerical error", f(epsilon))
    return epsilon

In [36]:
bennett_epsilon(50,300,20,0.05)

Numerical error -4.5102810375396984e-17


13.457174445041813

In [37]:
from ipywidgets import FloatSlider
@interact 
def concentration_300(n=IntSlider(value=1,min=50,max=5000,step=50),sigma=FloatSlider(value=10, min=1,max=300,step=1)):
    X = np.minimum(np.abs(70+6*np.random.normal(size=(n))),300)
    means = np.mean(X,axis=0)
    epsilon = np.sqrt(-1/(2*n/(300**2))*np.log((1-0.95)/2))
    epsilon2 = bennett_epsilon(n,b=300,sigma=sigma,alpha=0.05)
    print("95%% confidence interval rough [%.2f, %.2f] for n=%d" % (means-epsilon,means+epsilon,n))
    print("95%% confidence interval Bennett [%.2f, %.2f] for n=%d" % (means-epsilon2,means+epsilon2,n))

interactive(children=(IntSlider(value=50, description='n', max=5000, min=50, step=50), FloatSlider(value=10.0,…